# 20N Newsgroup

In [47]:
from xml.dom import minidom
import os, nltk, re
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
import numpy as np

In [54]:
characters_to_remove = '!()#@~,."><*'
pattern = "[" + characters_to_remove + "]"
p = PorterStemmer()
tokenizer = nltk.RegexpTokenizer(r'\w+')

In [49]:
def process(p, tokenizer, text):
    """ Applies standard pre-processing to given text.
    
    Args:
        p (gensim.parsing.porter.PorterStemmer): stemmer object.
        tokenizer (nltk.tokenize.regexp.RegexpTokenizer): tokenizr object.
        text (str): text to preprocess.
    
    Returns:
        list: preprocessed text.
    
    """
    # Converts to lowercase
    doc_nor = text.lower()
    
    # Removes stopwords
    doc_sw = remove_stopwords(doc_nor)
    
    # Stems text
    doc_stem = p.stem_sentence(doc_sw)
    
    # Lemmatizes text
    # TODO: Lemmatizer
    
    # Returns preprocessed text
    return tokenizer.tokenize(doc_stem)

In [55]:
categories = os.listdir('./data/20news')

In [59]:
listed_text = []
listed_categories = []
for category in categories:
    files = os.listdir('./data/20news/' + category)
    for file in files:
        doc = open('./data/20news/' + category + '/' + file, encoding = 'ISO-8859-1',mode='r')
        text = re.sub(pattern, "", doc.read().replace('\n', '').replace('  ', ''))
        listed_text.append(process(p, tokenizer, text))
        listed_categories.append(text)
        doc.close()

In [60]:
dictionary = corpora.Dictionary(listed_text)
dictionary.save('vocab.dict')
doc_corpus = []
for doc in listed_text:
    doc_corpus.append(dictionary.doc2bow(doc))

In [65]:
# pending to remove 1 freq word... los correos si se les quita el arroba hacen estragos. creo que es preferible
# quitarlos

[['from',
  'wlsmithvalveheartrriuwoca',
  'wayn',
  'smithsubject',
  're',
  'id',
  'vs',
  'scsiin',
  'articl',
  '1qpu0uinnbt1dns1nmsuedu',
  'bgrubbdantenmsuedu',
  'grubb',
  'writes',
  'wlsmithvalveheartrriuwoca',
  'wayn',
  'smith',
  'writes',
  'sinc',
  'mac',
  'us',
  'scsi',
  '1',
  'hard',
  'drive',
  'ye',
  'figur',
  'includ',
  'ahundr',
  'scsi',
  'driversthi',
  'sloppi',
  'peopl',
  'dumbwhat',
  'group',
  'this',
  'thi',
  'mac',
  'groupok',
  'scsi',
  'spec',
  'list',
  'whi',
  'spec',
  'list',
  'again',
  'w',
  'talk',
  'scsi',
  'pc',
  'mac',
  'ora',
  'unix',
  'boxand',
  'talk',
  'isa',
  'bu',
  'possibl',
  'eisa',
  'vlbthi',
  'isin',
  't',
  'compperiphsscsitel',
  'perform',
  'figur',
  'singl',
  'scsi',
  'drive',
  'pcwith',
  'isa',
  'eisa',
  'vlb',
  'bustheoret',
  'perform',
  'figur',
  'relev',
  'group',
  'thisdebatei',
  'm',
  'sure',
  'platform',
  'canhandl',
  '40',
  'megs',
  'sec',
  'scsi',
  'xyz',
  'wid